In [34]:
import numpy as np
import pandas as pd
import umap.umap_ as umap
import hdbscan
import cv2
import matplotlib.pyplot as plt
import os
import plotly.graph_objects as go
from tqdm import tqdm
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from scipy.cluster.hierarchy import fcluster
from sklearn.metrics import silhouette_score
from bayes_opt import BayesianOptimization
import glob

In [20]:
new_column_names = [
    'Time', 'Hip x', 'Hip y', 'Hip likelihood',
    'Knee x', 'Knee y', 'Knee likelihood',
    'Ankle x', 'Ankle y', 'Ankle likelihood',
    'Heel (ventral) x', 'Heel (ventral) y', 'Heel (ventral) likelihood',
    'Instep (dorsal) x', 'Instep (dorsal) y', 'Instep (dorsal) likelihood',
    'Toe 1 (Proximal Thumb) x', 'Toe 1 (Proximal Thumb) y', 'Toe 1 (Proximal Thumb) likelihood',
    'Toe2 x', 'Toe2 y', 'Toe2 likelihood',
    'Toe 3 (Middle) x', 'Toe 3 (Middle) y', 'Toe 3 (Middle) likelihood',
    'Toe4 x', 'Toe4 y', 'Toe4 likelihood',
    'Toe 5 (Distal Pinky) x', 'Toe 5 (Distal Pinky) y', 'Toe 5 (Distal Pinky) likelihood',
]
def process_data(file_path):
    # Read only the first row to check column names
    df_header = pd.read_csv(file_path, nrows=0)
    
    if list(df_header.columns) != new_column_names:
        df = pd.read_csv(file_path, skiprows=3, header=None)
        df.columns = new_column_names
    else:
        df = pd.read_csv(file_path)

    coordinate_columns = [col for col in df.columns if ' x' in col or ' y' in col]
    df = df[(df[coordinate_columns] != 0).all(axis=1)]

    return df

In [21]:
def load_and_scale_data(batch_folder, video_format, skiprows, sep, decimal, batch_number):
    folder_path = f"{batch_folder}/*.{video_format}"
    print(folder_path)
    csv_files = sorted(glob.glob(folder_path.replace(f".{video_format}", ".csv")))
    print(csv_files)
    video_files = sorted(glob.glob(folder_path))
    dataframes = []

    for file_id, file in enumerate(csv_files):   
        df = process_data(file)
        coord_columns = [col for col in df.columns if ' x' in col or ' y' in col]
        scaler = StandardScaler()
        scaled_coords = scaler.fit_transform(df[coord_columns])
        scaled_df = pd.DataFrame(scaled_coords, columns=coord_columns)
        for col in df.columns:
            if col not in coord_columns:
                scaled_df[col] = df[col]
        scaled_df['FileID'] = file_id
        scaled_df['Batch'] = batch_number
        dataframes.append(scaled_df)

    return pd.concat(dataframes).reset_index(drop=True), video_files, csv_files

In [22]:
batch_folder = "E:/UofT/ECE2500/batch10"  # Replace with the actual path
video_format = "mp4"  
skiprows = 0  
sep = ","  
decimal = "."  
batch_number = 10

# Run the function
scaled_data, video_files, csv_files = load_and_scale_data(
    batch_folder, video_format, skiprows, sep, decimal, batch_number
)
scaled_data

E:/UofT/ECE2500/batch10/*.mp4
['E:/UofT/ECE2500/batch10\\TIS Sweep 1_100-1726266331DLC_resnet50_batch_10Sep21shuffle1_150000_filtered.csv', 'E:/UofT/ECE2500/batch10\\TIS Sweep 1_99-1726264847DLC_resnet50_batch_10Sep21shuffle1_150000_filtered.csv', 'E:/UofT/ECE2500/batch10\\TIS Sweep 2_100-1726266894DLC_resnet50_batch_10Sep21shuffle1_150000_filtered.csv', 'E:/UofT/ECE2500/batch10\\TIS Sweep 2_99-1726267369DLC_resnet50_batch_10Sep21shuffle1_150000_filtered.csv']


,Hip x,Hip y,Knee x,Knee y,Ankle x,Ankle y,Heel (ventral) x,Heel (ventral) y,Instep (dorsal) x,Instep (dorsal) y,...,Ankle likelihood,Heel (ventral) likelihood,Instep (dorsal) likelihood,Toe 1 (Proximal Thumb) likelihood,Toe2 likelihood,Toe 3 (Middle) likelihood,Toe4 likelihood,Toe 5 (Distal Pinky) likelihood,FileID,Batch
0,0.737044,-1.314555,-0.575300,-1.098360,-2.274634,-2.267028,-2.365389,-2.256856,2.056392,-2.282445,...,0.999928,0.999661,0.999645,0.999971,0.999903,0.999949,0.999935,0.999867,0,10
1,1.335380,-1.314555,-0.020567,-1.035038,-2.263529,-2.267028,-2.331807,-2.231137,2.056392,-2.282445,...,0.999923,0.999680,0.999662,0.999974,0.999876,0.999943,0.999947,0.999840,0,10
2,1.498894,-1.314555,0.120572,-1.035038,-2.236884,-2.201349,-2.305519,-2.231137,2.056392,-2.277585,...,0.999922,0.999768,0.999689,0.999972,0.999891,0.999942,0.999947,0.999848,0,10
3,1.600806,-1.396708,0.120572,-0.725140,-2.236884,-2.256500,-2.331807,-2.231137,2.046760,-2.277585,...,0.999889,0.999650,0.999694,0.999975,0.999890,0.999942,0.999945,0.999838,0,10
4,1.654722,-1.396708,0.120572,-0.725140,-2.236884,-2.201349,-2.365389,-2.231137,2.046760,-2.277585,...,0.999899,0.999712,0.999702,0.999964,0.999897,0.999946,0.999950,0.999812,0,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168101,-0.057193,0.498051,-0.215460,0.625567,0.232447,0.771508,0.314568,0.204524,0.258195,0.622358,...,0.999334,0.999943,0.999871,0.999968,0.999864,0.999773,0.999841,0.999885,3,10
168102,-0.057193,0.498051,-0.215460,0.630449,0.232447,0.776801,0.314568,0.201334,0.261557,0.621141,...,0.999370,0.999942,0.999868,0.999964,0.999876,0.999776,0.999852,0.999871,3,10
168103,-0.124394,0.508909,-0.215460,0.631343,0.232447,0.792386,0.314568,0.204499,0.261557,0.621141,...,0.999366,0.999947,0.999859,0.999966,0.999872,0.999767,0.999871,0.999886,3,10
168104,-0.124394,0.498051,-0.215460,0.630449,0.232447,0.792386,0.313297,0.199930,0.261557,0.619374,...,0.999344,0.999937,0.999857,0.999966,0.999874,0.999779,0.999869,0.999878,3,10


In [40]:
def perform_umap_clustering(data, n_components, n_neighbors, min_dist):
    umap_embedding = umap.UMAP(n_neighbors=int(n_neighbors), min_dist=min_dist, n_components=int(n_components), random_state=42).fit_transform(data)
    return umap_embedding

In [52]:
def find_optimal_umap_and_clustering(data, use_hdbscan, n_neighbors_range, min_dist_range, n_components_range, n_clusters_range=None, min_cluster_size_range=None, min_samples_range=None):
    def umap_hdbscan_evaluate(n_neighbors, min_dist, n_components, min_cluster_size, min_samples):
        umap_embedding = perform_umap_clustering(data, n_components=n_components, n_neighbors=n_neighbors, min_dist=min_dist)
        clusterer = hdbscan.HDBSCAN(min_cluster_size=int(min_cluster_size), min_samples=int(min_samples))
        clusterer.fit(umap_embedding)
        labels = clusterer.labels_
        if len(set(labels)) > 1 and len(set(labels)) != 0:
            return silhouette_score(umap_embedding, labels)
        else:
            return -1

    def umap_kmeans_evaluate(n_neighbors, min_dist, n_components, n_clusters):
        umap_embedding = perform_umap_clustering(data, n_components=n_components, n_neighbors=n_neighbors, min_dist=min_dist)
        kmeans = KMeans(n_clusters=int(n_clusters), random_state=42)
        labels = kmeans.fit_predict(umap_embedding)
        if len(set(labels)) > 1:
            return silhouette_score(umap_embedding, labels)
        else:
            return -1

    best_score = -1
    best_params = {}
    best_labels = None
    best_umap_embedding = None

    if use_hdbscan:
        pbounds = {
            'n_neighbors': (min(n_neighbors_range), max(n_neighbors_range)),
            'min_dist': (min(min_dist_range), max(min_dist_range)),
            'n_components': (min(n_components_range), max(n_components_range)),
            'min_cluster_size': (min(min_cluster_size_range), max(min_cluster_size_range)),
            'min_samples': (min(min_samples_range), max(min_samples_range))
        }
        optimizer = BayesianOptimization(
            f=umap_hdbscan_evaluate,
            pbounds=pbounds,
            random_state=42
        )
        optimizer.maximize(init_points=2, n_iter=30)

        best_params = optimizer.max['params']
        best_umap_embedding = perform_umap_clustering(data, n_components=best_params['n_components'], n_neighbors=best_params['n_neighbors'], min_dist=best_params['min_dist'])
        clusterer = hdbscan.HDBSCAN(min_cluster_size=int(best_params['min_cluster_size']), min_samples=int(best_params['min_samples']))
        clusterer.fit(best_umap_embedding)
        best_labels = clusterer.labels_
        best_score = optimizer.max['target']

    else:
        pbounds = {
            'n_neighbors': (min(n_neighbors_range), max(n_neighbors_range)),
            'min_dist': (min(min_dist_range), max(min_dist_range)),
            'n_components': (min(n_components_range), max(n_components_range)),
            'n_clusters': (min(n_clusters_range), max(n_clusters_range))
        }
        optimizer = BayesianOptimization(
            f=umap_kmeans_evaluate,
            pbounds=pbounds,
            random_state=42
        )
        optimizer.maximize(init_points=2, n_iter=30)

        best_params = optimizer.max['params']
        best_umap_embedding = perform_umap_clustering(data, n_components=best_params['n_components'], n_neighbors=best_params['n_neighbors'], min_dist=best_params['min_dist'])
        kmeans = KMeans(n_clusters=int(best_params['n_clusters']), random_state=42)
        best_labels = kmeans.fit_predict(best_umap_embedding)
        best_score = optimizer.max['target']

    print(f"Best UMAP and Clustering Parameters: {best_params}")
    print(f"Best Silhouette Score: {best_score}")
    return best_umap_embedding, best_labels, best_params

In [25]:
def get_cluster_sequences(df, labels):
    df['Cluster'] = labels
    cluster_sequences = {}
    for cluster in df['Cluster'].unique():
        if cluster == -1:  # Skip noise points
            continue
        cluster_data = df[df['Cluster'] == cluster]
        sequences = []
        current_sequence = []
        previous_time = None
        for _, row in cluster_data.iterrows():
            time_value = row['Time']
            if previous_time is None or time_value == previous_time + 1:
                current_sequence.append(row)
            else:
                if current_sequence:
                    sequences.append(pd.DataFrame(current_sequence))
                current_sequence = [row]
            previous_time = time_value
        
        if current_sequence:
            sequences.append(pd.DataFrame(current_sequence))
        
        # Check assigned FileIDs in the sequences
        for seq in sequences:
            print(f"Cluster {cluster}, FileID in sequence: {seq['FileID'].iloc[0]}")
        
        cluster_sequences[cluster] = sequences
    return cluster_sequences

In [26]:
def create_video(sequence, output_filename, video_files, fps):
    batch_number = int(sequence['Batch'].iloc[0])
    file_id = int(sequence['FileID'].iloc[0])
    
    # Check for valid FileID within range
    if file_id >= len(video_files) or file_id < 0:
        print(f"Error: FileID {file_id} is out of range for the available video files.")
        return

    video_file = video_files[file_id]
    print(f"Creating video from file: {video_file}")
    
    cap = cv2.VideoCapture(video_file)
    if not cap.isOpened():
        print(f"Error: Cannot open video file {video_file}")
        return

    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    out = cv2.VideoWriter(output_filename, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))
    empty_frames = 0
    
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    for time_index in sequence['Time']:
        if time_index >= 0 and time_index < total_frames:
            cap.set(cv2.CAP_PROP_POS_FRAMES, time_index)
            ret, frame = cap.read()
            if ret:
                out.write(frame)
            else:
                empty_frames += 1
                print(f"Frame at {time_index} in file {video_file} could not be read.")
        else:
            empty_frames += 1
            print(f"Invalid frame index {time_index} for file {video_file}. Total frames: {total_frames}")
    
    if empty_frames > 0:
        print(f"Video {output_filename} has {empty_frames} frames that could not be read.")
    
    cap.release()
    out.release()
    print(f'Video saved at: {output_filename}')


In [27]:
def combine_cluster_videos(cluster_folder, combined_output_filename, fps):
    video_files = sorted(glob.glob(os.path.join(cluster_folder, '*.mp4')))
    if not video_files:
        print(f"No videos found in {cluster_folder} to combine.")
        return

    print(f"Combining videos from {cluster_folder} into {combined_output_filename}")
    first_video_file = video_files[0]
    cap = cv2.VideoCapture(first_video_file)
    if not cap.isOpened():
        print(f"Error: Cannot open video file {first_video_file}")
        return

    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    cap.release()

    out = cv2.VideoWriter(combined_output_filename, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

    for video_file in video_files:
        cap = cv2.VideoCapture(video_file)
        if not cap.isOpened():
            print(f"Error: Cannot open video file {video_file}")
            continue
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            out.write(frame)
        cap.release()
    out.release()
    print(f"Combined video saved at: {combined_output_filename}")

In [49]:
def process_batch(batch_folder, video_format, skiprows, sep, decimal, batch_number, use_hdbscan, n_neighbors_range, min_dist_range, n_components_range, n_clusters_range, min_cluster_size_range, min_samples_range, output_folder, fps):
    data, video_files, csv_files = load_and_scale_data(batch_folder, video_format, skiprows, sep, decimal, batch_number)
    umap_embedding, labels, best_params = find_optimal_umap_and_clustering(data.select_dtypes(include=[np.number]), use_hdbscan, n_neighbors_range, min_dist_range, n_components_range, n_clusters_range, min_cluster_size_range, min_samples_range)
    
    optimal_num_clusters = best_params.get('n_clusters', len(set(labels)) - (1 if -1 in labels else 0))  # Use the optimal number of clusters

    # Update cluster sequences based on the optimal number of clusters
    cluster_sequences = get_cluster_sequences(data, labels)
    clustering_method = "HDBSCAN" if use_hdbscan else "KMeans"
    batch_output_folder = os.path.join(output_folder, f'batch_{batch_number}', f'Umap+{clustering_method}')
    if not os.path.exists(batch_output_folder):
        os.makedirs(batch_output_folder)

    max_sequence_length = max(len(sequences) for sequences in cluster_sequences.values())
    regrouped_sequences = [[] for _ in range(max_sequence_length)]
    for cluster, sequences in cluster_sequences.items():
        for i, sequence in enumerate(sequences):
            regrouped_sequences[i].append(sequence)

    # Create videos 
    for i, sequences in enumerate(regrouped_sequences):
        if i >= optimal_num_clusters:  
            break
        regrouped_folder = os.path.join(batch_output_folder, f'cluster_{i}')
        if not os.path.exists(regrouped_folder):
            os.makedirs(regrouped_folder)
        for j, sequence in enumerate(sequences):
            output_filename = os.path.join(regrouped_folder, f'cluster_{i}_sequence_{j+1}.mp4')
            create_video(sequence, output_filename, video_files, fps)

    for i in range(optimal_num_clusters):
        regrouped_folder = os.path.join(batch_output_folder, f'cluster_{i}')
        combined_output_filename = os.path.join(batch_output_folder, f'cluster_{i}_combined_sequences.mp4')
        combine_cluster_videos(regrouped_folder, combined_output_filename, fps)

In [50]:
process_batch(
    batch_folder = "E:/UofT/ECE2500/batch1",  
    video_format = "avi",  
    skiprows = 0,  
    sep = ",",  
    decimal = ".",  
    batch_number = 1,
    use_hdbscan=True,
    n_neighbors_range=[5, 300],    
    min_dist_range=[0, 1],
    n_components_range=[2, 11],
    n_clusters_range=range(2, 10),
    min_cluster_size_range=[5, 100],
    min_samples_range=[1, 100],  
    output_folder='E:/UofT/ECE2500/video/batch1',
    fps=15
)


E:/UofT/ECE2500/batch1/*.avi
['E:/UofT/ECE2500/batch1\\120_kSa_3p3_V_-340mV_offsetDLC_resnet50_ece2500yJan17shuffle1_100000_filtered.csv', 'E:/UofT/ECE2500/batch1\\30_kSa_3p3_V_-340mV_offsetDLC_resnet50_ece2500yJan17shuffle1_100000_filtered.csv', 'E:/UofT/ECE2500/batch1\\90_kSa_3p3_V_-340mV_offsetDLC_resnet50_ece2500yJan17shuffle1_100000_filtered.csv', 'E:/UofT/ECE2500/batch1\\9_kSa_3p3_V_-340mV_offsetDLC_resnet50_ece2500yJan17shuffle1_100000_filtered.csv']
|   iter    |  target   | min_cl... | min_dist  | min_sa... | n_comp... | n_neig... |
-------------------------------------------------------------------------------------


e:\Anaconda1\envs\env_ml\Lib\site-packages\umap\umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


| 1         | 0.1935    | 40.58     | 0.9507    | 73.47     | 7.388     | 51.03     |


e:\Anaconda1\envs\env_ml\Lib\site-packages\umap\umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


| 2         | 0.2174    | 19.82     | 0.05808   | 86.75     | 7.41      | 213.9     |


...

| 101       | 0.3059    | 86.02     | 0.9437    | 82.79     | 9.22      | 75.47     |


e:\Anaconda1\envs\env_ml\Lib\site-packages\umap\umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


| 102       | 0.4104    | 92.29     | 0.766     | 52.29     | 10.56     | 216.3     |


e:\Anaconda1\envs\env_ml\Lib\site-packages\umap\umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


Best UMAP and Clustering Parameters: {'min_cluster_size': 50.40422105735845, 'min_dist': 0.07053342736147672, 'min_samples': 75.29917648412412, 'n_components': 2.764193328040683, 'n_neighbors': 272.6695041939408}
Best Silhouette Score: 0.5330919027328491
Cluster 0, FileID in sequence: 0.0
Cluster 0, FileID in sequence: 1.0
Cluster 0, FileID in sequence: 2.0
Cluster 0, FileID in sequence: 3.0
Cluster 1, FileID in sequence: 0.0
Cluster 1, FileID in sequence: 1.0
Cluster 1, FileID in sequence: 2.0
Cluster 1, FileID in sequence: 3.0
Creating video from file: E:/UofT/ECE2500/batch1\120_kSa_3p3_V_-340mV_offset.avi
Video saved at: E:/UofT/ECE2500/video/batch1\batch_1\Umap+HDBSCAN\cluster_0\cluster_0_sequence_1.mp4
Creating video from file: E:/UofT/ECE2500/batch1\120_kSa_3p3_V_-340mV_offset.avi
Video saved at: E:/UofT/ECE2500/video/batch1\batch_1\Umap+HDBSCAN\cluster_0\cluster_0_sequence_2.mp4
Creating video from file: E:/UofT/ECE2500/batch1\30_kSa_3p3_V_-340mV_offset.avi
Video saved at: E:/Uo

In [ ]:
# Updated Test
process_batch(
    batch_folder = "E:/UofT/ECE2500/batch1",  
    video_format = "avi",  
    skiprows = 0,  
    sep = ",",  
    decimal = ".",  
    batch_number = 1,
    use_hdbscan=True,
    n_neighbors_range=[10, 30, 50, 80, 100, 150, 180],    
    min_dist_range=[0.1, 0.3, 0.5, 0.8],
    n_components_range=[2, 3, 4, 5, 6, 7, 8, 9, 10],
    n_clusters_range=range(2, 10),
    min_cluster_size_range=[5, 10, 20, 30, 50],
    min_samples_range=[ 5, 10, 20, 30, 50],  
    output_folder='E:/UofT/ECE2500/video/batch1',
    fps=15
)


In [ ]:
process_batch(
    batch_folder = "/Users/shawnyang/Desktop/UofT/ECE2500Y/ece2500Y/csv_coordinates/batch1",  
    video_format = "avi",  
    skiprows = 0,  
    sep = ",",  
    decimal = ".",  
    batch_number = 1,
    use_hdbscan=True,
    n_neighbors_range=[30, 50, 80, 100, 150],
    min_dist_range=[0.1, 0.5, 0.8],
    n_components_range=[3, 4, 5, 6, 7, 8],
    n_clusters_range=range(2, 10),
    min_cluster_size_range=[10, 30, 50],
    min_samples_range=[10, 20, 30],
    output_folder='/Users/shawnyang/Desktop/UofT/ECE2500Y/ece2500Y/Analysis/animation/new/batch1',
    fps=15,
    # resolution=(640, 480)
)

# Batch2

In [ ]:
# Test
process_batch(
    batch_folder = "/Users/shawnyang/Desktop/UofT/ECE2500Y/ece2500Y/csv_coordinates/batch2",  
    video_format = "mp4",  
    skiprows = 0,  
    sep = ",",  
    decimal = ".",  
    batch_number = 2,
    use_hdbscan=False,
    n_neighbors_range=[30, 50, 100],    
    min_dist_range=[0.1, 0.5],
    n_components_range=[2, 3, 4, 5],
    n_clusters_range=range(2, 10),
    min_cluster_size_range=[10,20],
    min_samples_range=[10, 20],
    output_folder='/Users/shawnyang/Desktop/UofT/ECE2500Y/ece2500Y/Analysis/animation/new/batch2',
    fps=15,
    # resolution=(640, 480)
)

# Batch3

In [644]:
process_batch(
    batch_folder = "/Users/shawnyang/Desktop/UofT/ECE2500Y/ece2500Y/csv_coordinates/batch3",  
    video_format = "mp4",  
    skiprows = 0,  
    sep = ",",  
    decimal = ".",  
    batch_number = 3,
    use_hdbscan=True,
    n_neighbors_range=[30, 50, 100, 150],    
    min_dist_range=[0.1,0.3, 0.5],
    n_components_range=[2, 3, 4, 5],
    n_clusters_range=range(2, 8),
    min_cluster_size_range=[10,20,30],
    min_samples_range=[5,10, 20],
    output_folder='/Users/shawnyang/Desktop/UofT/ECE2500Y/ece2500Y/Analysis/animation/new/batch3',
    fps=15,
    # resolution=(640, 480)
)

/Users/shawnyang/Desktop/UofT/ECE2500Y/ece2500Y/csv_coordinates/batch3/*.mp4
['/Users/shawnyang/Desktop/UofT/ECE2500Y/ece2500Y/csv_coordinates/batch3/Biphasic_120_Hz_channel_1_aDLC_resnet50_ECE2500Y-third_batchMar13shuffle2_200000_filtered.csv', '/Users/shawnyang/Desktop/UofT/ECE2500Y/ece2500Y/csv_coordinates/batch3/Biphasic_120_Hz_channel_1_bDLC_resnet50_ECE2500Y-third_batchMar13shuffle2_200000_filtered.csv', '/Users/shawnyang/Desktop/UofT/ECE2500Y/ece2500Y/csv_coordinates/batch3/Biphasic_120_Hz_channel_6_aDLC_resnet50_ECE2500Y-third_batchMar13shuffle2_200000_filtered.csv', '/Users/shawnyang/Desktop/UofT/ECE2500Y/ece2500Y/csv_coordinates/batch3/Biphasic_120_Hz_channel_6_bDLC_resnet50_ECE2500Y-third_batchMar13shuffle2_200000_filtered.csv', '/Users/shawnyang/Desktop/UofT/ECE2500Y/ece2500Y/csv_coordinates/batch3/Biphasic_30_Hz_channel_1_aDLC_resnet50_ECE2500Y-third_batchMar13shuffle2_200000_filtered.csv', '/Users/shawnyang/Desktop/UofT/ECE2500Y/ece2500Y/csv_coordinates/batch3/Biphasic_30

/Users/shawnyang/miniconda/envs/tensorflow/lib/python3.9/site-packages/umap/umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")
/Users/shawnyang/miniconda/envs/tensorflow/lib/python3.9/site-packages/umap/umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")
/Users/shawnyang/miniconda/envs/tensorflow/lib/python3.9/site-packages/umap/umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")
/Users/shawnyang/miniconda/envs/tensorflow/lib/python3.9/site-packages/umap/umap_.py:1945: UserWarning: n_job

Best UMAP and Clustering Parameters: {'n_neighbors': 30, 'min_dist': 0.3, 'n_components': 3, 'min_cluster_size': 10, 'min_samples': 20}
Cluster 1, FileID in sequence: 0.0
Cluster 1, FileID in sequence: 1.0
Cluster 1, FileID in sequence: 2.0
Cluster 1, FileID in sequence: 3.0
Cluster 1, FileID in sequence: 4.0
Cluster 1, FileID in sequence: 5.0
Cluster 1, FileID in sequence: 6.0
Cluster 1, FileID in sequence: 7.0
Cluster 1, FileID in sequence: 8.0
Cluster 1, FileID in sequence: 9.0
Cluster 1, FileID in sequence: 10.0
Cluster 1, FileID in sequence: 11.0
Cluster 1, FileID in sequence: 12.0
Cluster 1, FileID in sequence: 13.0
Cluster 1, FileID in sequence: 14.0
Cluster 1, FileID in sequence: 15.0
Cluster 0, FileID in sequence: 4.0
Creating video from file: /Users/shawnyang/Desktop/UofT/ECE2500Y/ece2500Y/csv_coordinates/batch3/Biphasic_120_Hz_channel_1_a.mp4
Video saved at: /Users/shawnyang/Desktop/UofT/ECE2500Y/ece2500Y/Analysis/animation/new/batch3/batch_3/Umap+HDBSCAN/cluster_0/cluster_0

In [ ]:
process_batch(
    batch_folder = "/Users/shawnyang/Desktop/UofT/ECE2500Y/ece2500Y/csv_coordinates/batch3",  
    video_format = "mp4",  
    skiprows = 0,  
    sep = ",",  
    decimal = ".",  
    batch_number = 3,
    use_hdbscan=True,
    n_neighbors_range=[30, 50, 100, 150],    
    min_dist_range=[0.1,0.3, 0.5],
    n_components_range=[2, 3, 4, 5],
    n_clusters_range=range(2, 8),
    min_cluster_size_range=[10,20,30],
    min_samples_range=[5,10, 20],
    output_folder='/Users/shawnyang/Desktop/UofT/ECE2500Y/ece2500Y/Analysis/animation/new/batch3',
    fps=15,
    # resolution=(640, 480)
)

# Batch4

In [ ]:
process_batch(
    batch_folder = "/Users/shawnyang/Desktop/UofT/ECE2500Y/ece2500Y/csv_coordinates/batch4",  
    video_format = "mp4",  
    skiprows = 0,  
    sep = ",",  
    decimal = ".",  
    batch_number = 4,
    use_hdbscan=True,
    n_neighbors_range=[30, 50, 100, 150],    
    min_dist_range=[0.1,0.3, 0.5],
    n_components_range=[2, 3, 4, 5],
    n_clusters_range=range(2, 8),
    min_cluster_size_range=[10,20,30],
    min_samples_range=[5,10, 20],
    output_folder='/Users/shawnyang/Desktop/UofT/ECE2500Y/ece2500Y/Analysis/animation/new/batch4',
    fps=15,
    # resolution=(640, 480)
)

# Batch9

In [ ]:
process_batch(
    batch_folder = "/Users/shawnyang/Desktop/UofT/ECE2500Y/ece2500Y/csv_coordinates/batch9",  
    video_format = "mp4",  
    skiprows = 0,  
    sep = ",",  
    decimal = ".",  
    batch_number = 4,
    use_hdbscan=True,
    n_neighbors_range=[30, 50, 100, 150],    
    min_dist_range=[0.1, 0.3, 0.5],
    n_components_range=[2, 3, 4, 5],
    n_clusters_range=range(2, 8),
    min_cluster_size_range=[10,20,30],
    min_samples_range=[5,10, 20],
    output_folder='/Users/shawnyang/Desktop/UofT/ECE2500Y/ece2500Y/Analysis/animation/new/batch9',
    fps=15,
    # resolution=(640, 480)
)

In [ ]:
process_batch(
    batch_folder = "/Users/shawnyang/Desktop/UofT/ECE2500Y/ece2500Y/csv_coordinates/batch9",  
    video_format = "mp4",  
    skiprows = 0,  
    sep = ",",  
    decimal = ".",  
    batch_number = 9,
    use_hdbscan=False,  
    n_neighbors_range=[30, 50, 100, 150],    
    min_dist_range=[0.1, 0.3, 0.5],
    n_components_range=[2, 3, 4, 5],
    n_clusters_range=range(2, 8),
    min_cluster_size_range=[10,20,30],
    min_samples_range=[5,10, 20],
    output_folder='/Users/shawnyang/Desktop/UofT/ECE2500Y/ece2500Y/Analysis/animation/new/batch9',
    fps=15,
    # resolution=(640, 480)
)

# Batch10

In [ ]:
process_batch(
    batch_folder = "E:/UofT/ECE2500/batch10",  
    video_format = "mp4",  
    skiprows = 0,  
    sep = ",",  
    decimal = ".",  
    batch_number = 10,
    use_hdbscan=True,
    n_neighbors_range=[5, 250],    
    min_dist_range=[0, 0.8],
    n_components_range=[2, 8],
    n_clusters_range=range(2, 10),
    min_cluster_size_range=[5, 50],
    min_samples_range=[1, 50],  
    output_folder='E:/UofT/ECE2500/video/batch10',
    fps=15
)


E:/UofT/ECE2500/batch10/*.mp4
['E:/UofT/ECE2500/batch10\\TIS Sweep 1_100-1726266331DLC_resnet50_batch_10Sep21shuffle1_150000_filtered.csv', 'E:/UofT/ECE2500/batch10\\TIS Sweep 1_99-1726264847DLC_resnet50_batch_10Sep21shuffle1_150000_filtered.csv', 'E:/UofT/ECE2500/batch10\\TIS Sweep 2_100-1726266894DLC_resnet50_batch_10Sep21shuffle1_150000_filtered.csv', 'E:/UofT/ECE2500/batch10\\TIS Sweep 2_99-1726267369DLC_resnet50_batch_10Sep21shuffle1_150000_filtered.csv']
|   iter    |  target   | min_cl... | min_dist  | min_sa... | n_comp... | n_neig... |
-------------------------------------------------------------------------------------


e:\Anaconda1\envs\env_ml\Lib\site-packages\umap\umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


| 1         | -0.01124  | 21.85     | 0.7606    | 36.87     | 5.592     | 43.22     |


e:\Anaconda1\envs\env_ml\Lib\site-packages\umap\umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


| 2         | 0.1773    | 12.02     | 0.04647   | 43.44     | 5.607     | 178.5     |


e:\Anaconda1\envs\env_ml\Lib\site-packages\umap\umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


In [ ]:
process_batch(
    batch_folder = "E:/UofT/ECE2500/batch10",  
    video_format = "mp4",  
    skiprows = 0,  
    sep = ",",  
    decimal = ".",  
    batch_number = 10,
    use_hdbscan=True,
    n_neighbors_range=[30, 50, 100, 150],    
    n_components_range=[2, 3, 4, 5],
    n_clusters_range=range(2, 8),
    min_cluster_size_range=[10,20,30],
    output_folder='E:/UofT/ECE2500/video/batch10',
    fps=15,
    # resolution=(640, 480)
)